# Othello AI

## By: Jackson, Ava, Jack, Eugene

In [68]:
import random
import math
import sys


EMPTY, BLACK, WHITE, OUTER = '.', '@', 'o', '?'
PIECES = (EMPTY, BLACK, WHITE, OUTER)
PLAYERS = {BLACK: 'Black', WHITE: 'White'}

UP, DOWN, LEFT, RIGHT = -10, 10, -1, 1
UP_RIGHT, DOWN_RIGHT, DOWN_LEFT, UP_LEFT = -9, 11, 9, -11
DIRECTIONS = (UP, UP_RIGHT, RIGHT, DOWN_RIGHT, DOWN, DOWN_LEFT, LEFT, UP_LEFT)


def squares():
    return [i for i in range(11, 89) if 1 <= (i % 10) <= 8]


def initial_board():
    board = [OUTER] * 100
    for i in squares():
        board[i] = EMPTY

    board[44], board[45] = WHITE, BLACK
    board[54], board[55] = BLACK, WHITE
    return board


def print_board(board):
    rep = ''
    rep += '  %s\n' % ' '.join(map(str, range(1, 9)))
    for row in range(1, 9):
        begin, end = 10 * row + 1, 10 * row + 9
        rep += '%d %s\n' % (row, ' '.join(board[begin:end]))
    return rep


def is_valid(move):
    return isinstance(move, int) and move in squares()


def opponent(player):
    return BLACK if player is WHITE else WHITE


def find_bracket(square, player, board, direction):
    bracket = square + direction
    if board[bracket] == player:
        return None
    opp = opponent(player)
    while board[bracket] == opp:
        bracket += direction
    return None if board[bracket] in (OUTER, EMPTY) else bracket


def is_legal(move, player, board):
    hasbracket = lambda direction: find_bracket(move, player, board, direction)
    return board[move] == EMPTY and any(map(hasbracket, DIRECTIONS))


def make_move(move, player, board):
    board[move] = player
    for d in DIRECTIONS:
        make_flips(move, player, board, d)
    print("\n" + "-" * 40 + "\n")
    text_player = "White" if player == "o" else "Black"
    print("{}'s move: ({}, {})\n".format(text_player, str(move)[0], str(move)[1]))
    print(print_board(board))
    return board


def make_flips(move, player, board, direction):
    bracket = find_bracket(move, player, board, direction)
    if not bracket:
        return
    square = move + direction
    while square != bracket:
        board[square] = player
        square += direction


class IllegalMoveError(Exception):
    def __init__(self, player, move, board):
        self.player = player
        self.move = move
        self.board = board

    def __str__(self):
        return '%s cannot move to square %d' % (PLAYERS[self.player], self.move)


def legal_moves(player, board):
    return [sq for sq in squares() if is_legal(sq, player, board)]


def any_legal_move(player, board):
    return any(is_legal(sq, player, board) for sq in squares())


def play(black_strategy, white_strategy):
    board = initial_board()
    print(print_board(board))
    player = BLACK
    strategy = lambda who: black_strategy if who == BLACK else white_strategy
    while player is not None:
        move = get_move(strategy(player), player, board)
        make_move(move, player, board)
        player = next_player(board, player)

    return board, score(BLACK, board)


def next_player(board, prev_player):
    opp = opponent(prev_player)
    if any_legal_move(opp, board):
        return opp
    elif any_legal_move(prev_player, board):
        print("No available moves")
        return prev_player
    return None


def get_move(strategy, player, board):
    copy = list(board)  # copy the board to prevent cheating
    move = strategy(player, copy)
    if not is_valid(move) or not is_legal(move, player, board):
        raise IllegalMoveError(player, move, copy)
    return move


def score(player, board):
    mine, theirs = 0, 0
    opp = opponent(player)
    for sq in squares():
        piece = board[sq]
        if piece == player:
            mine += 1
        elif piece == opp:
            theirs += 1

    return mine - theirs


def return_winner(score):
    print("Score: " + str(score))
    if score > 0:
        print("Black wins")
    elif score == 0:
        print("Draw")
    else:
        print("White wins")


def random_strategy(player, board):
    return random.choice(legal_moves(player, board))


def real_player(player, board):
    print("Your move")
    print("Available moves:")
    moves = legal_moves(player, board)
    for i in moves:
        print("{}) {}, {}".format(moves.index(i) + 1, str(i)[0], str(i)[1]))
    choice = input("Where would you like to go? (Choose an option)")
    while True:
        if choice.isdigit() and int(choice) - 1 < len(moves) and int(choice) != 0:
            choice = int(choice) - 1
            return moves[choice]
        else:
            print("Not a valid move.")
            choice = input("Where would you like to go? (Choose an option)")


# ava's greedy strategy
def corner_move(move):
    return move == 11 or move == 18 or move == 81 or move == 88


def greedy(player, board):
    moves = legal_moves(player, board)
    corner_moves = []
    # set bestMove to the first legal move to start
    bestMove = moves[0]
    # check if there are corner moves
    for move in moves:
        if corner_move(move):
            corner_moves.append(move)
    if corner_moves != []:
        # best move is the first available corner move
        bestMove = corner_moves[0]
        return bestMove
    else:
        bestScore = -1
        for move in moves:
            make_invisible_move(move, player, board)
            move_score = score(player, board)
            if move_score >= bestScore:
                bestScore = move_score
                bestMove = move
        return bestMove


# Start of mcts strategy
class TreeNode:
    def __init__(self, player, board, action):
        self.children = []
        self.parent = None
        self.board = board
        self.player = player
        self.action = action
        self.legal_moves = legal_moves(player, board)
        self.untried = self.legal_moves
        self.reward = 0
        self.visits = 0

    def add_child(self, tn):
        self.children.append(tn)
        tn.parent = self


def make_invisible_move(move, player, board):
    copy = list(board)
    copy[move] = player
    for d in DIRECTIONS:
        make_flips(move, player, copy, d)
    return copy


def invisible_next_player(board, prev_player):
    opp = opponent(prev_player)
    if any_legal_move(opp, board):
        return opp
    elif any_legal_move(prev_player, board):
        return prev_player
    return None


def tree_policy(tn):
    while len(tn.legal_moves) > 0:
        if len(tn.untried) > 0:
            return expand(tn)
        else:
            # best_child second parameter?
            tn = best_child(tn)
    return tn


def default_policy(my_player, tn):
    player = tn.player
    board = tn.board
    moves = legal_moves(player, board)
    move_count = 0
    while len(moves) > 0:
        move_count += 1
        action = random.choice(moves)
        board = make_invisible_move(action, player, board)
        player = invisible_next_player(board, player)
        moves = legal_moves(player, board)
    if score(my_player, board) > 0:
        return 100
    else:
        return 0


def expand(tn):
    action = random.choice(tn.untried)
    child = TreeNode(tn.player, make_invisible_move(action, tn.player, tn.board), action)
    tn.add_child(child)
    return child


def upper_confidence(tn):
    return tn.reward / tn.visits + 1 / math.sqrt(2) * (math.sqrt(2 * math.log(tn.parent.visits)) / tn.visits)


def best_child(tn):
    best_child = None
    for i in tn.children:
        # Pick highest reward, which had it, and return the one with the best reward
        if best_child == None or upper_confidence(i) > upper_confidence(best_child):
            best_child = i
    return best_child


def backup_negamax(tn, reward):
    while tn != None:
        tn.visits += 1
        tn.reward += reward
        if tn.parent != None and tn.parent.player != tn.player:
            reward = -reward
        tn = tn.parent


def update_progress(progress):
    barLength = 20
    status = ""
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
        status = "error: progress var must be float\r\n"
    if progress < 0:
        progress = 0
        status = "Halt...\r\n"
    if progress >= 1:
        progress = 1
        status = "Done...\r\n"
    block = int(round(barLength*progress))
    text = "\rProgress: [{0}] {1}% {2}".format( "#"*block + "-"*(barLength-block), progress*100, status)
    sys.stdout.write(text)
    sys.stdout.flush()


def mcts(player, board):
    root = TreeNode(player, board, None)
    progress = 0
    iterations = 1000
    for i in range(iterations):
        if i % (iterations/10) == 0:
            progress += 0.1
            update_progress(progress)
        child = tree_policy(root)
        reward = default_policy(player, child)
        backup_negamax(child, reward)
    return best_child(root).action


def greedy_mcts(player, board):
    for i in legal_moves(player, board):
        if corner_move(i):
            return i
    return mcts(player, board)


# Currently implemented strategies: mcts, random_strategy, real_player, greedy, greedy_mcts
print("Note: the game works by inputting a row first and then a column.\n")
(board, points) = play(real_player, greedy_mcts)
return_winner(points)

Note: the game works by inputting a row first and then a column.

  1 2 3 4 5 6 7 8
1 . . . . . . . .
2 . . . . . . . .
3 . . . . . . . .
4 . . . o @ . . .
5 . . . @ o . . .
6 . . . . . . . .
7 . . . . . . . .
8 . . . . . . . .

Your move
Available moves:
1) 3, 4
2) 4, 3
3) 5, 6
4) 6, 5
Where would you like to go? (Choose an option)1

----------------------------------------

Black's move: (3, 4)

  1 2 3 4 5 6 7 8
1 . . . . . . . .
2 . . . . . . . .
3 . . . @ . . . .
4 . . . @ @ . . .
5 . . . @ o . . .
6 . . . . . . . .
7 . . . . . . . .
8 . . . . . . . .

Progress: [####################] 99.99999999999999% 
----------------------------------------

White's move: (5, 3)

  1 2 3 4 5 6 7 8
1 . . . . . . . .
2 . . . . . . . .
3 . . . @ . . . .
4 . . . @ @ . . .
5 . . o o o . . .
6 . . . . . . . .
7 . . . . . . . .
8 . . . . . . . .

Your move
Available moves:
1) 6, 2
2) 6, 3
3) 6, 4
4) 6, 5
5) 6, 6
Where would you like to go? (Choose an option)3

----------------------------------------